In [ ]:
from pavlov import *
monitor()

In [ ]:
from pavlov import stats
stats.plot(['2021-01-05 18-37-29 gentle-mouth 7x7', '*7x7-no-league*', '*hot-bumps 7x7*'], 'sample-rate.learner', skip=1)

In [ ]:
from pavlov import archive
archive.view('*medium-sails*', 'boardlaw/networks.py')